# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [60]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [61]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,18.09,63,75,2.06,CA,1724008666
1,1,ilulissat,69.2167,-51.1000,6.01,70,75,3.09,GL,1724008667
2,2,tura,25.5198,90.2201,23.29,98,79,2.01,IN,1724008668
3,3,kingston,17.9970,-76.7936,33.53,66,40,7.72,JM,1724008669
4,4,albany,42.6001,-73.9662,23.68,80,100,4.00,US,1724008510


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [63]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity'
    )

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [64]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] < 40) & (city_data_df['Max Temp'] > 10)]
city_data_df = city_data_df.loc[(city_data_df['Wind Speed'] < 15) & (city_data_df['Wind Speed'] > 5)]
city_data_df = city_data_df.loc[(city_data_df['Humidity'] < 50) & (city_data_df['Humidity'] > 30)]
city_data_df = city_data_df.loc[(city_data_df['Cloudiness'] < 80) & (city_data_df['Cloudiness'] > 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
85,85,timbuktu,16.7735,-3.0074,32.82,46,45,5.74,ML,1724008771
136,136,whitehorse,60.7161,-135.0538,22.28,43,40,5.66,CA,1724008831
158,158,minas de marcona,-15.2119,-75.1103,18.73,48,2,6.18,PE,1724008859
248,248,wajir,1.7471,40.0573,28.40,40,52,11.81,KE,1724008973
299,299,garoowe,8.4054,48.4845,30.01,37,6,9.00,SO,1724009035
394,394,burao,9.5221,45.5336,25.85,43,8,10.17,SO,1724009150
400,400,kapuskasing,49.4169,-82.4331,20.05,49,75,6.17,CA,1724009157
504,504,essaouira,31.5125,-9.7700,32.72,45,4,6.17,MA,1724009283


### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
85,timbuktu,ML,16.7735,-3.0074,46,
136,whitehorse,CA,60.7161,-135.0538,43,
158,minas de marcona,PE,-15.2119,-75.1103,48,
248,wajir,KE,1.7471,40.0573,40,
299,garoowe,SO,8.4054,48.4845,37,
394,burao,SO,9.5221,45.5336,43,
400,kapuskasing,CA,49.4169,-82.4331,49,
504,essaouira,MA,31.5125,-9.7700,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 500000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
timbuktu - nearest hotel: Hotel La Maison
whitehorse - nearest hotel: Town & Mountain Hotel
minas de marcona - nearest hotel: No hotel found
wajir - nearest hotel: Kilimanjaro
garoowe - nearest hotel: Red Sea
burao - nearest hotel: Liiban hotel burco ( ENG:Shibiin )
kapuskasing - nearest hotel: Travelodge
essaouira - nearest hotel: Essaouira Wind Palace


,City,Country,Lat,Lng,Humidity,Hotel Name
85,timbuktu,ML,16.7735,-3.0074,46,Hotel La Maison
136,whitehorse,CA,60.7161,-135.0538,43,Town & Mountain Hotel
158,minas de marcona,PE,-15.2119,-75.1103,48,No hotel found
248,wajir,KE,1.7471,40.0573,40,Kilimanjaro
299,garoowe,SO,8.4054,48.4845,37,Red Sea
394,burao,SO,9.5221,45.5336,43,Liiban hotel burco ( ENG:Shibiin )
400,kapuskasing,CA,49.4169,-82.4331,49,Travelodge
504,essaouira,MA,31.5125,-9.7700,45,Essaouira Wind Palace


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    hover_cols=['Hotel Name','Country']
    )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)